# Naive Bayes Intuition

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import math

from sklearn import datasets
from sklearn.model_selection import train_test_split

Naive bayes is Naive coz it doesn't care about word order.\
High bias, low variance\
Features are independent.\
Continous Values are bad.

https://www.youtube.com/watch?v=O2L2Uv9pdDA

$P(A|B) = \frac{P(B|A)*P(A)}{P(B)}$

$P(y|X) = \frac{P(X|y)*P(y)}{P(X)}$

$P(y|X) = \frac{P(X_1|y)*P(X_2|y)*...*P(X_n|y)*P(y)}{P(X)}$

denomiator is ignored

In [19]:
def printAll(*ps) : 
    [print(p) for p in ps]

printAll(
np.argmax([1,1,2,5]),
np.unique([1,1,2,5]),
)

3
[1 2 5]


In [2]:
X, y = datasets.make_classification(n_samples=8, n_features=10, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.shape, len(y_train)

((5, 10), 5)

## Step 1: Create Matrices

In [3]:
n_samples, n_features = X_train.shape
classes = np.unique(y_train)
n_classes = len(classes)
n_classes

2

In [4]:
mean = np.zeros((n_classes, n_features), dtype=np.float64)
var = np.zeros((n_classes, n_features), dtype=np.float64)
priors = np.zeros(n_classes, dtype=np.float64)
mean.shape

(2, 10)

In [5]:
X_train[y_train==0].shape

(3, 10)

In [8]:
len(X_train[y_train==0])/n_samples, len(X_train[y_train==1])/n_samples

(0.6, 0.4)

## Step 2: Cal Col Mean & Var

In [9]:
for i, c in enumerate(classes):
    X_c = X_train[y_train==c]
    # Mean & Variance for each feature
    mean[i,:] = X_c.mean(axis=0)
    var[i,:] = X_c.var(axis=0)
    # Relative frequencies of classes as Priors
    priors[i] = (X_c.shape[0]/n_samples)
    print(i, X_c.shape, X_c.mean(axis=0).shape, priors[i])

0 (3, 10) (10,) 0.6
1 (2, 10) (10,) 0.4


### PDF

In [15]:
def pdf(i, x):
    # Returns list of pdfs for each feature
    m = mean[i]
    v = var[i]
    numerator = np.exp(-((x - m) ** 2) / (2 * v))
    denominator = np.sqrt(2 * np.pi * v)
    return numerator / denominator

np.sum(np.log(pdf(1, [5,1,4,5,6,7,5,4,1,2])))

C:\Users\timashan\AppData\Local\Temp\ipykernel_18736\1797519152.py:9: RuntimeWarning: divide by zero encountered in log
  np.sum(np.log(pdf(1, [5,1,4,5,6,7,5,4,1,2])))


-inf

In [48]:
numerator = np.exp(-((np.array([5,4,1,-8]) - 5) ** 2) / (2 * 25))
denominator = np.sqrt(2 * np.pi * 25)
pdf_0 = numerator / denominator
pdf_0
# sns.kdeplot(np.random.normal(5, 5, 1000))

array([0.07978846, 0.07820854, 0.05793831, 0.00271659])

In [57]:
from scipy.stats import norm
print(norm.pdf([5,4,1,-8], 5, 5))

[0.07978846 0.07820854 0.05793831 0.00271659]


## Step 3: Calc Posterior with Gaussian PDF

In [18]:
pred = []
for i0, x in enumerate(X_test):
    poss = []
    for i, c in enumerate(classes):
        prior = np.log(priors[i])
        pos = np.sum(np.log(pdf(i, x))) # likelihood
        # Logs so we sum em up
        pos = pos + prior
        poss.append(pos)
        print(i0, c,  round(prior, 1), round(pos, 1))
    print(poss, np.argmax(poss))
    pred.append(classes[np.argmax(poss)])

len(pred)

0 0 -0.5 -588.4
0 1 -0.9 -36.7
[-588.3624617843499, -36.67956053183627] 1
1 0 -0.5 -92.2
1 1 -0.9 -38.4
[-92.24526151715004, -38.405524948946855] 1
2 0 -0.5 -inf
2 1 -0.9 -168.5
[-inf, -168.47784497714176] 1


C:\Users\timashan\AppData\Local\Temp\ipykernel_18736\1350139866.py:6: RuntimeWarning: divide by zero encountered in log
  pos = np.sum(np.log(pdf(i, x))) # likelihood


3

In [19]:
np.sum(y_test == pred)/len(pred)

0.6666666666666666